## Summary notes

Perform a stratified analyses on the results of a stratified case-control study.

Data were taken from investigating the possible association between alcohol consumption and fatal car accidents in New York (J.R. McCarroll and W. Haddon Jr, 1962).
The data were stratified by marital status, which was believed to be a possible confounder.
The *exposure* was blood alcohol level of 100mg% or greater;
*Cases* were drivers who were killed in car accidents for which they were considered to be responsible;
and *controls* were selected drivers passing the locations where the accidents of the cases occurred, at the same time of day and on the same day of the week.

The results were as follows.

Level: *Married*

|                      | fatality (+)   | no fatality (-) |
| -------------------- | -------------: | --------------: |
| **over 100mg% (+)**  | 4              | 5               |
| **under 100mg% (-)** | 5              | 103             |

Level: *Not married*

|                      | fatality (+)    | no fatality (-) |
| -------------------- | --------------: | --------------: |
| **over 100mg% (+)**  | 10              | 3               |
| **under 100mg% (-)** | 5               | 43              |

The analysis was undertaken using StatsModels.

The data was stored on remotely, so we defined a function `cache_file` to handle the retrieval of the file.

The main difficulty with a stratified analysis is that we need to deal with the data in many different shapes, so we defined two functions that return the data in the needed shapes.
Two further functions were defined to handle a pieces of analysis.[^1]

The level, exposure, and case-control labels were stored as three `list`s, with variables named *levels*,[^6] *exposures*, and *casecons*.
The order of the elements in *exposures* and *diseases* lists is important:
`Table2x2`[^2] requires the data to be some sequence with shape `(2, 2)`:

|                     | disease (+) | no diease (-) |
| --------------------| ----------: | ------------: |
| **exposed (+)**     | *a*         | *b*           |
| **not exposed (-)** | *c*         | *d*           |

And `StratifiedTable`[^3] requires the data to be some sequence with shape `(2, 2, 2)`:

| level   | exposure           | disease (+) | no diease (-) |
| ------- |--------------------| ----------: | ------------: |
| **one** | **exposed (+)**    | *a*         | *b*           |
|         |**not exposed (-)** | *c*         | *d*           |
| **two** | **exposed (+)**    | *e*         | *f*           |
|         |**not exposed (-)** | *g*         | *h*           |

As such, the labels should be ordered:

- *exposures* = (*exposed*, *not exposed*)
- *casecons* = (*case*, *control*)

The data were cached and used to a initialise a Pandas `DataFrame`.

The main difficulty with a stratified analysis is that we need to deal with the data in many different shapes.
Three variables were initialised to hold the different views of the data.
These were:

- *level_tables*, a collection of `Table2x2` instances, where each instance represents a contingency table for a specific level
- *aggregated_table*, an instance of `Table2x2` using the aggregated data
  - Aggregated by *exposure*, *casecon*
- *strat_table*, an instance of `StratifiedTable`

Measures of association were calculated:

- Level-specific odds ratios
- Unadjusted odds ratios[^4]
- Adjusted odds ratios[^5]
  
And two hypothesis tets were performed:

- Test of no association
- Test of homogeneity

These topics are covered in M249, Book 1, Part 2.

## Dependencies

In [18]:
import os
import requests
from dataclasses import dataclass
import numpy as np
import pandas as pd
from statsmodels import api as sm

## Functions

In [19]:
def cache_file(url: str, fname: str, dir_: str = './__cache') -> str:
    """Cache the file at given url in the given dir_ with the given
    fname and return the local path.

    Preconditions:
    - dir_ exists
    """
    local_path = f'{dir_}/{fname}'
    if fname not in os.listdir(dir_):
        r = requests.get(url, allow_redirects=True)
        open(local_path, 'wb').write(r.content)
    return local_path

## Main

### Initialise the labels

In [20]:
levels = np.array(['married', 'not married'])
exposures = np.array(['over 100mg', 'under 100mg'])
casecons = np.array(['fatality', 'no fatality'])

### Cache the data

In [21]:
path = cache_file(
        url=('https://raw.githubusercontent.com/ljk233/laughingrook-datasets'
             + '/main/m249/medical/drinkdriving.csv'),
        fname='drinkdriving.csv'
)

### Load the data

Use the cached file to initialise a `DataFrame`.

In [22]:
data = pd.read_csv(path)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   n         8 non-null      int64 
 1   level     8 non-null      object
 2   exposure  8 non-null      object
 3   casecon   8 non-null      object
dtypes: int64(1), object(3)
memory usage: 384.0+ bytes


In [23]:
data

,n,level,exposure,casecon
0,4,married,over 100mg,fatality
1,5,married,over 100mg,no fatality
2,5,married,under 100mg,fatality
3,103,married,under 100mg,no fatality
4,10,not married,over 100mg,fatality
5,3,not married,over 100mg,no fatality
6,5,not married,under 100mg,fatality
7,43,not married,under 100mg,no fatality


### Prepare the data

Initialise a new `DataFrame` using *data*, with the columns *level*, *exposure*, *disease* as ordered `Categorical` variables to ensure that the data will be sorted as expected.

In [24]:
cat_data = pd.DataFrame().assign(
    n=data['n'].to_numpy(),
    level=pd.Categorical(data['level'], levels, ordered=True),
    exposure=pd.Categorical(data['exposure'], exposures, ordered=True),
    casecon=pd.Categorical(data['casecon'], casecons, ordered=True)
).sort_values(
    by=['level', 'exposure', 'casecon']
)
cat_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   n         8 non-null      int64   
 1   level     8 non-null      category
 2   exposure  8 non-null      category
 3   casecon   8 non-null      category
dtypes: category(3), int64(1)
memory usage: 524.0 bytes


Output pivot tables with marginal totals.

In [25]:
for level in levels:
    print(
        cat_data.query(
            'level == @level'
        ).pivot_table(
            values='n',
            index=['level', 'exposure'],
            columns='casecon',
            aggfunc='sum',
            margins=True,
            margins_name='subtotal'
        ).query(
            "level in [@level, 'subtotal']"
        ),
        '\n'
    )

casecon               fatality  no fatality  subtotal
level    exposure                                    
married  over 100mg          4            5       9.0
         under 100mg         5          103     108.0
subtotal                     9          108     117.0 

casecon                  fatality  no fatality  subtotal
level       exposure                                    
not married over 100mg         10            3      13.0
            under 100mg         5           43      48.0
subtotal                       15           46      61.0 



### Initialise the contingency tables

Initialise contingency tables for different views of the data.

Variable *level_tables* is collection of level-specific 2x2 contingency tables.

In [26]:
_get_table = lambda df, level: (
    sm.stats.Table2x2(
        df.query('level == @level')['n'].to_numpy().reshape((2, 2))
    )
)
level_tables = [_get_table(cat_data, level) for level in levels]
for table, level in zip(level_tables, levels):
    print(f'level={level}\n{table}\n')

level=married
A 2x2 contingency table with counts:
[[  4.   5.]
 [  5. 103.]]

level=not married
A 2x2 contingency table with counts:
[[10.  3.]
 [ 5. 43.]]



Variable *aggregated_table* is a 2x2 contingency table of the aggregated data.

In [27]:
aggregated_table = sm.stats.Table2x2(
    cat_data.groupby(['exposure', 'casecon'])
            .sum()
            .to_numpy()
            .reshape((2, 2))
)
print(aggregated_table)

A 2x2 contingency table with counts:
[[ 14.   8.]
 [ 10. 146.]]


Variable *strat_table* is a 2x2x2 contingency table that is stratified by level.

In [28]:
strat_table = (
    sm.stats.StratifiedTable(
                data.sort_values(by=['exposure', 'casecon', 'level'])['n']
                    .to_numpy()
                    .reshape((2, 2, 2))
    )
)
print(strat_table.table)

[[[  4.  10.]
  [  5.   3.]]

 [[  5.   5.]
  [103.  43.]]]


### Odds ratios

Return point and interval estimates of the level-specific, unadjusted, and adjusted odds ratios.

Return the level-specific odds ratios.

In [29]:
_get_measure = lambda table: (
    np.array([table.oddsratio,
              table.oddsratio_confint()[0],
              table.oddsratio_confint()[1]]
    )
)
pd.DataFrame(
    data=[_get_measure(table).round(5) for table in level_tables],
    columns=['point', 'lcb', 'ucb'],
    index=pd.Index(levels, name='level')
)

,point,lcb,ucb
level,,,
married,16.48000,3.35421,80.96998
not married,28.66667,5.85662,140.31607


Return the unadjusted odds ratio.

In [30]:
pd.Series(
    data=[aggregated_table.oddsratio,
          aggregated_table.oddsratio_confint()[0],
          aggregated_table.oddsratio_confint()[1]],
    index=['point', 'lcb', 'ucb'],
    name='unadjusted odds ratio'
).round(
    5
)

point    25.55000
lcb       8.68217
ucb      75.18883
Name: unadjusted odds ratio, dtype: float64

Return the adjusted odds ratio.

In [31]:
pd.Series(
    data=[strat_table.oddsratio_pooled,
           strat_table.oddsratio_pooled_confint()[0],
           strat_table.oddsratio_pooled_confint()[1]],
    index=['point', 'lcb', 'ucb'],
    name='adjusted odds ratio'
).round(
    3
)

point    23.001
lcb       7.465
ucb      70.866
Name: adjusted odds ratio, dtype: float64

### Hypothesis testing

Return the results of a test of no association and test of homogeneity.

In [32]:
_r = strat_table.test_null_odds(correction=True)
pd.Series(
    data=[_r.statistic, _r.pvalue],
    index=['statistic', 'pvalue'],
    name='test of no association'
).round(
    3
)

statistic    36.604
pvalue        0.000
Name: test of no association, dtype: float64

In [33]:
_r = strat_table.test_equal_odds(adjust=True)
pd.Series(
    data=[_r.statistic, _r.pvalue],
    index=['statistic', 'pvalue'],
    name='test of homogeneity'
).round(
    3
)

statistic    0.236
pvalue       0.627
Name: test of homogeneity, dtype: float64

## References

McCarroll, J.R. and Haddon Jr, W., 1962. A controlled study of fatal automobile accidents in New York City. Journal of chronic diseases, 15(8), pp.811-826.

In [34]:
%load_ext watermark
%watermark --iv

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
sys        : 3.10.6 (tags/v3.10.6:9c7b4bd, Aug  1 2022, 21:53:49) [MSC v.1932 64 bit (AMD64)]
requests   : 2.28.1
statsmodels: 0.13.2
numpy      : 1.23.2
pandas     : 1.4.3



[^1]: These are convenience functions to prevent repetition of complicated `DataFrame` constructions
[^2]: See [statsmodels.stats.contingency_tables.Table2x2](https://www.statsmodels.org/v0.13.0/generated/statsmodels.stats.contingency_tables.Table2x2.html)
[^3]: See [statsmodels.stats.contingency_tables.StratifiedTable](https://www.statsmodels.org/v0.13.0/generated/statsmodels.stats.contingency_tables.StratifiedTable.html)
[^4]: This is the crude odds ratio
[^5]: This is the Mantel-Haenszel odds ratio
[^6]: The order of the levels array is not particularly important, but we left it here to be consistent throughout